In [1]:
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv("C:\\Users\\chira\\Downloads\\diabetes (2).csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
df.duplicated().sum()

0

In [7]:
x=df.drop('Outcome', axis=1)
y=df['Outcome']

In [8]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()


In [9]:
x_scaled=scaler.fit_transform(x)

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [11]:
model=Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(8,))) #input layer with 8 neurons
model.add(layers.Dense(1, activation='sigmoid')) #output layer with 1 neuron

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(x_scaled, y, epochs=10, batch_size=10, validation_split=0.2)

Epoch 1/10


62/62 [==============================] - 1s 4ms/step - loss: 0.7922 - accuracy: 0.6156 - val_loss: 0.7137 - val_accuracy: 0.6234
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 0.7185 - accuracy: 0.6010 - val_loss: 0.6656 - val_accuracy: 0.6234
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 0.6680 - accuracy: 0.6042 - val_loss: 0.6309 - val_accuracy: 0.6299
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 0.6282 - accuracy: 0.6254 - val_loss: 0.6050 - val_accuracy: 0.6494
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 0.5963 - accuracy: 0.6352 - val_loss: 0.5827 - val_accuracy: 0.6753
Epoch 6/10
62/62 [==============================] - 0s 3ms/step - loss: 0.5688 - accuracy: 0.6710 - val_loss: 0.5659 - val_accuracy: 0.6753
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 0.5477 - accuracy: 0.7036 - val_loss: 0.5502 - val_accuracy: 0.7078
Epoch 8/10
62/62 [

In [13]:
import keras_tuner as kt

In [14]:
def build_model(hp):
    model=Sequential()
    model.add(layers.Dense(units=hp.Int('units', min_value=8, max_value=128, step=8), activation='relu', input_shape=(8,)))
    model.add(layers.Dense(1, activation='sigmoid'))

    optimized=hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adagrad'])

    model.compile(optimizer=optimized, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [15]:
tuner=kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='my_dir', project_name='intro_to_kt')

Reloading Tuner from my_dir\intro_to_kt\tuner0.json


In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x_scaled, y, test_size=0.2, random_state=42)

In [17]:
x_train

array([[-0.54791859, -1.15469351, -3.57259724, ..., -4.06047387,
        -0.50700636, -1.04154944],
       [ 1.53084665, -0.27837344,  0.66661825, ..., -0.48135115,
         2.44666971,  1.4259954 ],
       [-0.84488505,  0.56664949, -1.19450074, ..., -0.41789153,
         0.55003518, -0.95646168],
       ...,
       [ 1.82781311, -0.62264204,  0.87340925, ...,  1.72704372,
         2.00573238,  0.40494237],
       [-1.14185152,  0.62924378, -3.57259724, ...,  1.32090213,
        -0.8059981 , -0.36084741],
       [-1.14185152,  0.12848945,  1.39038675, ..., -1.20479085,
        -0.63385134, -1.04154944]])

In [18]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

In [19]:
tuner.results_summary()

Results summary
Results in my_dir\intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
units: 80
optimizer: rmsprop
Score: 0.7792207598686218

Trial 2 summary
Hyperparameters:
units: 72
optimizer: sgd
Score: 0.7575757503509521
Traceback (most recent call last):
  File "c:\Users\chira\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\chira\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chira\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_m

In [20]:
tuner.get_best_hyperparameters(num_trials=1)[0].values

{'units': 80, 'optimizer': 'rmsprop'}

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 81 (324.00 Byte)
Trainable params: 81 (324.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
def build_model(hp):
    model=Sequential()
    model.add(layers.Dense(units=hp.Int('units', min_value=8, max_value=128, step=8), activation='relu', input_shape=(8,)))
    model.add(layers.Dense(1, activation='sigmoid'))



    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [23]:
tuner=kt.RandomSearch(build_model, objective='val_accuracy',max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [24]:
tuner.search(x_scaled, y, epochs=5, validation_split=0.2, validation_data=(x_test, y_test))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 81 (324.00 Byte)
Trainable params: 81 (324.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
tuner.get_best_hyperparameters(num_trials=1)[0].values

{'units': 96}

In [27]:
model=tuner.hypermodel.build(tuner.get_best_hyperparameters(num_trials=1)[0])

In [28]:
model.fit(x_scaled, y, epochs=10, batch_size=10, validation_split=0.2)

Epoch 1/10
62/62 [==============================] - 0s 3ms/step - loss: 0.5532 - accuracy: 0.7410 - val_loss: 0.5238 - val_accuracy: 0.7922
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 0.4923 - accuracy: 0.7704 - val_loss: 0.4946 - val_accuracy: 0.7727
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 0.4711 - accuracy: 0.7850 - val_loss: 0.4830 - val_accuracy: 0.7922
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 0.4593 - accuracy: 0.7834 - val_loss: 0.4783 - val_accuracy: 0.7792
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 0.4530 - accuracy: 0.7752 - val_loss: 0.4773 - val_accuracy: 0.7792
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 0.4501 - accuracy: 0.7834 - val_loss: 0.4792 - val_accuracy: 0.7857
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 0.4453 - accuracy: 0.7915 - val_loss: 0.4794 - val_accuracy: 0.7597
Epoch 8/10
62/62 [==

In [29]:
def build_model(hp):
    model=Sequential()
    model.add(layers.Dense(96, activation='relu', input_shape=(8,))) #input layer with 8 neurons and 96 neurons in first hidden layer

    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=8, max_value=128, step=8), activation='relu'))

    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [30]:
tuner=kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='my_dir', project_name='intro_to_kt')

Reloading Tuner from my_dir\intro_to_kt\tuner0.json


In [31]:
tuner.search(x_scaled, y, epochs=10, validation_split=0.2)

In [32]:
tuner.get_best_hyperparameters(num_trials=1)[0].values

{'units': 80, 'optimizer': 'rmsprop'}

In [34]:
tuner.search(x_scaled, y, epochs=10, validation_split=0.2)
# tuner.get_best_models(num_models=1)[0]

In [36]:
tuner.get_best_hyperparameters(num_trials=1)[0].values

{'units': 80, 'optimizer': 'rmsprop', 'num_layers': 1, 'units_0': 8}

In [39]:
tuner._build_hypermodel(hp=tuner.get_best_hyperparameters(num_trials=1)[0])
tuner.get_best_hyperparameters(num_trials=1)[0].values

{'units': 80, 'optimizer': 'rmsprop', 'num_layers': 1, 'units_0': 8}